# Tutorial for Analyzing LMS Files

## Purpose
This notebook is designed to help analysts understand and access the files created by the Ed-Fi LMS Toolkit extractor utilities.

## Pandas

We'll be using [Pandas](https://pandas.pydata.org/), which is the industry standard tool for high performance data analysis in Python. If you're new to Python and Pandas and want to do some additional reading, then you might be interested in these two books by Jake VanderPlas, both available for free:

* [A Whirlwind Tour of Python](https://jakevdp.github.io/WhirlwindTourOfPython/)
* [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)

## Getting Started

This notebook was developed with Python 3.8 and might not work with earlier versions. The source code directory for this notebook contains a `poetry.lock` and `pyproject.toml` file; these are used by [Poetry](https://python-poetry.org/) to manage resource dependencies. Ideally you will install Poetry and then run `poetry install` from a command prompt to load required resources. If you would like to run without using Poetry, then you can manually load dependencies using Pip:

```bash
# Optional step if NOT using poetry
pip install pandas
pip install ipykernel
pip install jupyter --user
```

It is recommended that these be installed in a virtual environment, which Poetry handles for you. Alternately, you can use equivalent commands in Anaconda.

This notebook can be run from within Visual Studio Code or it can run in a browser window by executing the following command:

```bash
# With Poetry
poetry run jupyter notebook

# Without Poetry
jupyter notebook
```

## Understanding the Filesystem

The LMS Extractors output a number of discrete files, corresponding to concepts in the Ed-Fi LMS Unified Data Model:

* Activities
* Assignments
* Attendance Events (_only with Schoology_)
* Grades
* Sections
* Section Associations (aka _enrollments_)
* Submissions
* User Activities
* Users

Each file contains all of the current data for the given model, so that you only need to read one file to get a complete snapshot for a single resource. But there is a catch to that: some of these concepts are section-specific, for example assignments. Rather than try to store all assignment data _for all sections_ in a single file, we create _one assignment file per section_. The same is true for attendance events, grades, section associations, and user activities. Furthermore, submissions are dependent on assignments, thus there is one submissions file per assignment, containing all submission records for that assignment. The file layout mirrors this heirarchy: there is a directory for each resource type, and dependent resources are nested under directories named for the given section or assignment.

This convention may seem a little strange for a human, but it is very easy to navigate for the computer. Each directory may have multiple files, one for each time you run the extractor, but you only need to load the most recent file to get the current snapshot. We make that easy by using the date and time as the file name. Thus after running once, you may end up with files like this, where 12345 is the source system identifier for a unique section and 67890 is the source system identifier for a unique assignment:

![Sample file layout](filesystem.svg)

Note: `base_directory` is whatever directory was specified in the configuration when running the extractor utility.

Presumably the extractor utility will be run on a periodic basis, for instance weekly or daily. In that case each directory will have multiple files. Since the filenames have the date and time embedded in them, sorting on the file names will make it easy to pick up only the most recent file, regardless of whether or not some other process has modified the file and thus altered the operating system date on the file.

## Helper Functions

### Accessing Files

Outside this notebook you will find a Python module file called `lms_utilities.py`. This module contains a number of functions to help access files in the filesystem, including the following (where `base_directory` is the parent directory containing all of the LMS-extracted files):

| Function | Argument |
| -------- | -------- |
| get_users_file | base_directory (string) |
| get_sections_file | base_directory (string) |
| get_assignments_file | base_directory (string), section_id (int) |
| get_attendance_events_file | base_directory (string), section_id (int) |
| get_grades_file | base_directory (string), section_id (int) |
| get_section_associations_file | base_directory (string), section_id (int) |
| get_user_activities_file | base_directory (string), section_id (int) |
| get_submissions_file | base_directory (string), section_id (int), assignment_id (int) |

The next code cell demonstrates import and usage of these functions.

In [1]:
# Load this helper module to access the functions
import lms_utilities as lms

# Let's look at sample output from each filesystem function, using this 
# repository's `docs/sample-out` directory.
import os
sample_dir = os.path.join("..", "..", "docs", "sample-out")

print("Users file:", lms.get_users_file(sample_dir))
print("Sections file:", lms.get_sections_file(sample_dir))
print("Assignments file for Section 123456780:", lms.get_assignments_file(sample_dir, 123456780))
print("Assignment Submissions file for Assignment 2942251001:", lms.get_submissions_file(sample_dir, 123456780, 2942251001))
print("Attendance Events file for Section 2385758954:", lms.get_attendance_events_file(sample_dir, 2385758954))
print("Grades file for for Section 123456780:", lms.get_grades_file(sample_dir, 123456780))
print("Section Associations file for for Section 123456780:", lms.get_section_associations_file(sample_dir, 123456780))
print("User Activities file for Section 2385758954:", lms.get_user_activities_file(sample_dir, 2385758954))

Users file: ..\..\docs\sample-out\users\2020-09-18-15-05-01.csv
Sections file: ..\..\docs\sample-out\sections\2020-09-17-15-04-23.csv
Assignments file for Section 123456780: ..\..\docs\sample-out\section=123456780\assignments\2020-09-18-15-04-24.csv
Assignment Submissions file for Assignment 2942251001: ..\..\docs\sample-out\section=123456780\assignment=2942251001\submissions\2020-09-17-15-04-23.csv
Attendance Events file for Section 2385758954: ..\..\docs\sample-out\section=2385758954\attendance-events\2020-09-17-15-04-23.csv
Grades file for for Section 123456780: ..\..\docs\sample-out\section=123456780\grades\2020-09-18-15-04-24.csv
Section Associations file for for Section 123456780: ..\..\docs\sample-out\section=123456780\section-associations\2020-09-18-15-04-24.csv
User Activities file for Section 2385758954: ..\..\docs\sample-out\section=2385758954\user-activities\2020-09-21-11-08-34.csv


### Loading Files into DataFrames

The same `lms_utilities.py` file also contains a number of functions that will load the most recent CSV files - leveraging the functions above - into [Pandas DataFrames](https://pandas.pydata.org/) for display and analysis.

Note that the functions for data nested under sections is using the output from `get_all_sections`, and thus _will only pull data for active sections_. For example, if assignments, activities, etc. for section 12345 were retrieved historically, but section 12345 is no longer active (not in the current sections file), then these functions will ignore the data for section 12345. You can of course adapt these functions for your own use to access historical data as needed.

| Function Name | Arguments | Output |
| ------------- | --------- | ------ |
| get_all_users | base_directory (string) | DataFrame with all user records from the most recent file |
| get_all_sections | base_directory (string) | DataFrame with all section records from the most recent file |
| get_assignments | base_directory (string), section_id (int) | DataFrame with all assignments for the given section_id |
| get_all_assignments | base_directory (string), sections (DataFrame) | DataFrame with all assignments from all sections |
| get_attendance_events | base_directory (string), section_id (int) | DataFrame with all attendance events for the given section_id |
| get_all_attendance_events | base_directory (string), sections (DataFrame) | DataFrame with all attendance events from all sections |
| get_grades | base_directory (string), section_id (int) | DataFrame with all grades for the given section_id |
| get_all_grades | base_directory (string), sections (DataFrame) | DataFrame with all grades from all sections |
| get_section_associations | base_directory (string), section_id (int) | DataFrame with all section associations for the given section_id |
| get_all_section_associations | base_directory (string), sections (DataFrame) | DataFrame with all section associations from all sections |
| get_user_activities | base_directory (string), section_id (int) | DataFrame with all activities for the given section_id |
| get_all_user_activities | base_directory (string), sections (DataFrame) | DataFrame with all activities from all sections |
| get_submissions | base_directory (string), section_id (int), assignment_id(int) | DataFrame with all submissions for the given assignment |
| get_all_submissions | base_directory (string), assignments (DataFrame) | DataFrame with all submissions from all sections |

The next code cell demonstrates use of these functions by displaying the first few records for each type of file.

In [2]:
from IPython.display import display, Markdown

users_df = lms.get_all_users(sample_dir)

display(Markdown("### Users"))
display(users_df.head())

sections_df = lms.get_all_sections(sample_dir)

display(Markdown("### Sections"))
display(sections_df.head())

all_assignments_df = lms.get_all_assignments(sample_dir, sections_df)

display(Markdown("### Assignments"))
display(all_assignments_df.head())

all_attendance_events_df = lms.get_all_attendance_events(sample_dir, sections_df)

display(Markdown("### Attendance Events (Schoology only!)"))
display(all_attendance_events_df.head())

all_grades_df = lms.get_all_grades(sample_dir, sections_df)

display(Markdown("### Grades"))
display(all_grades_df.head())

all_section_associations_df = lms.get_all_section_associations(sample_dir, sections_df)

display(Markdown("### Section Associations"))
display(all_section_associations_df.head())

all_user_activities_df = lms.get_all_user_activities(sample_dir, sections_df)

display(Markdown("### User Activities"))
display(all_user_activities_df.head())

all_submissions_df = lms.get_all_submissions(sample_dir, all_assignments_df)

display(Markdown("### Submissions"))
display(all_submissions_df.head())

### Users

,SourceSystemIdentifier,SourceSystem,UserRole,LocalUserIdentifier,SISUserIdentifier,Name,EmailAddress,EntityStatus,CreateDate,LastModifiedDate
0,100032890,Schoology,student,mary.archer,604863,Mary Archer,Mary.Archer@studentgps.org,Archived,8/20/2020 12:34,9/18/2020 12:34
1,100032891,Schoology,student,kyle.hughes,604874,Kyle Hughes,Kyle.Hughes@studentgps.org,Archived,8/20/2020 12:34,9/18/2020 12:34
2,100032892,Schoology,student,peter.nash,604918,Peter Ivan Nash,Peter.Nash@studentgps.org,Active,8/20/2020 12:34,8/20/2020 12:34
3,100032893,Schoology,student,larry.mahoney,604927,Larry Mahoney,Larry.Mahoney@studentgps.org,Active,8/20/2020 12:34,8/20/2020 12:34
4,100032894,Schoology,student,roland.phillips,604938,Roland Phillips,Roland.Phillips@studentgps.org,Active,8/20/2020 12:34,8/20/2020 12:34


### Sections

,SourceSystemIdentifier,SourceSystem,SISSectionIdentifier,Title,SectionDescription,Term,LMSSectionStatus,EntityStatus,CreateDate,LastModifiedDate
0,123456780,Google Classroom,25590100102Trad220ALG112011,ALG-1,Algebra I,"255901001_2020_2019-2020_Fall,255901001_2020_2...",Archived,Active,8/20/2020 12:34,8/20/2020 12:34
1,123456789,Google Classroom,25590100101Trad120ENG112011,ENG-1,English/Language Arts I (9th grade),"255901001_2021_2020-2021_Fall,255901001_2021_2...",Active,Active,8/20/2020 12:34,8/20/2020 12:34
2,2385758954,Google Classroom,25590100102Trad220ALG112011,ALG-1,Algebra I,"255901001_2021_2020-2021_Fall,255901001_2021_2...",Active,Active,8/20/2020 12:34,8/20/2020 12:34
3,123456791,Google Classroom,NaN,ENG-STAFF-1,English language arts staff meeting,NaN,Unpublished,Active,9/3/2020 1:02,9/4/2020 0:01


### Assignments

...

,SourceSystemIdentifier,SourceSystem,EnrollmentStatus,StartDate,EndDate,LMSUserSourceSystemIdentifier,LMSSectionSourceSystemIdentifier,EntityStatus,CreateDate,LastModifiedDate
0,8123459,Canvas,Active,8/17/2020,12/18/2020,100032890,123456789,Active,8/20/2020 12:34,8/20/2020 12:34
1,8123456,Canvas,Completed,8/17/2020,12/18/2020,100032891,123456789,Active,8/20/2020 12:34,8/20/2020 12:34
2,8123457,Canvas,Inactive,8/17/2020,12/18/2020,100032892,123456789,Archived,8/20/2020 12:34,8/20/2020 12:34
3,8123458,Canvas,Active,8/17/2020,12/18/2020,100032893,123456789,Active,8/20/2020 12:34,8/20/2020 12:34
0,123456,Canvas,Active,1/4/2021,5/28/2021,100032894,123456780,Active,8/20/2020 12:34,8/20/2020 12:34


### User Activities

,SourceSystemIdentifier,SourceSystem,ActivityType,ActivityDateTime,ActivityStatus,MessagePost,TotalActivityTimeInMinutes,LMSSectionSourceSystemIdentifier,UserSourceSystemIdentifier,EntityStatus,CreateDate,LastModifiedDate
0,987654321,Canvas,Discussion,9/19/2020 14:23,Published,"First post!, with comma",98,34567,2385758954,Active,9/19/2020 14:23,9/19/2020 14:23


### Submissions

,SourceSystemIdentifier,SourceSystem,SubmissionStatus,SubmissionDateTime,EarnedPoints,Grade,AssignmentSourceSystemIdentifier,LMSUserSourceSystemIdentifier,EntityStatus,CreateDate,LastModifiedDate
0,456789156,Canvas,Returned,8/23/2020 23:59,77.0,C+,2942251001,2942251012,Active,8/20/2020 12:34,8/20/2020 12:34
1,456789157,Canvas,Returned,12/30/2020 23:59,82.0,B-,2942251001,100032894,Active,8/20/2020 12:34,8/20/2020 12:34
2,456789158,Canvas,Late,2021-05-01 02:00:00,91.0,A-,2942251001,100032897,Active,8/20/2020 12:34,8/20/2020 12:34
3,456789159,Canvas,Created,2021-04-31 04:32:00,NaN,NaN,2942251001,2942251012,Active,8/20/2020 12:34,8/20/2020 12:34
0,456789156,Canvas,Returned,8/23/2020 23:59,77.0,C+,2942251012,2942251012,Active,8/20/2020 12:34,8/20/2020 12:34


## Sample Analysis

### In Danger of Failing

Goal: Find all students with average grade (across all sections) below 70, displaying the student name and average grade.

This will require reading the sections file, the users file, section associations, and all grades file. These will need to be joined together, grouped by student, and then the grade will be calculated. Filter by average grade and then display the name and average grade columns.

But there are some "gotchas":

* Some of the grades are alphabetic! Interpret these on a scale like A = 95, B = 85, and so on.
* There are two different grade types: Final and Current. Filter on only Final grades for this exercise.

In [3]:
sample_dir = os.path.join("..", "..", "docs", "sample-out")

users = lms.get_all_users(sample_dir)
sections = lms.get_all_sections(sample_dir)
section_associations = lms.get_all_section_associations(sample_dir, sections)
grades = lms.get_all_grades(sample_dir, sections)

# Merge: users <--> section_associations <--> grades
left_key = "SourceSystemIdentifier"
right_key = "LMSUserSourceSystemIdentifier"
working = users.merge(section_associations, left_on=left_key, right_on=right_key, suffixes=("_u","_sa"))

left_key = "SourceSystemIdentifier_sa"
right_key = "LMSUserLMSSectionAssociationSourceSystemIdentifier"
working = working.merge(grades, left_on=left_key, right_on=right_key, suffixes=("","_g"))

# Filter on Final grades
working = working[working["GradeType"] == "Final"]

# Convert letter grades to numeric grades
def _convert_grades(grade):
    if grade.isnumeric():
        return grade
    
    switch = {
        "A": 95,
        "B": 85,
        "C": 75,
        "D": 65
    }
    return switch.get(grade, 55)

working["Grade"] = working["Grade"].apply(_convert_grades)

# Need to convert the Grade column to a numeric type - it still has the
# generic object type
working["Grade"] = working["Grade"].apply(float)

# Narrow down to columns we care about. For grouping, use the SIS unique 
# identifier instead of the name. The name will be lost in the grouping 
# anyway, and we'll have to re-merge with Users later on to recover it.
working = working[["Name", "SISUserIdentifier", "Grade"]]

# Group by student, using the SIS unique identifier
average_grades = working.groupby(by="SISUserIdentifier", as_index=False).mean(numeric_only=True)

# Pull the student names back in
average_grades = average_grades.merge(users[["Name","SISUserIdentifier"]], on="SISUserIdentifier")

# Filter to those in danger of failing
failing = average_grades[average_grades["Grade"] < 70]

# Finally, display the students of concern
display(Markdown("Students who are currently failing:"))
display(failing)

Students who are currently failing:

,SISUserIdentifier,Grade,Name
1,604927,62.0,Larry Mahoney


### Missing Assignment Submissions

Goal: find students who have not turned in assignment submissions.

Find the assignments that were due before a given date, and then enrich the data with the submission information. Drop records where there _is_ a _completed_ submission to focus only on those _without_ a completed submissions. Navigate backward from assignment to section to section association to user to get the students' names.

In [12]:
import pandas as pd

sample_dir = os.path.join("..", "..", "docs", "sample-out")

users = lms.get_all_users(sample_dir)
sections = lms.get_all_sections(sample_dir)
section_associations = lms.get_all_section_associations(sample_dir, sections)
assignments = lms.get_all_assignments(sample_dir, sections)
submissions = lms.get_all_submissions(sample_dir, assignments)

# The "DueDateTime" column is not automatically recognized as a datetime, so
# convert the data type
assignments["DueDateTime"] = pd.to_datetime(assignments["DueDateTime"])

# Filter assignments by those due before today
from datetime import datetime
assignments = assignments[assignments["DueDateTime"] < datetime.now()]

# Now let's combine some DataFrames to get a collection of all the students who 
# _should_ have turned in an assignment
key = "LMSSectionSourceSystemIdentifier"
working = assignments.merge(section_associations, on=key, suffixes=("","_sa"))

left_key = "LMSUserSourceSystemIdentifier"
right_key = "SourceSystemIdentifier"
working = working.merge(users, left_on=left_key, right_on=right_key, suffixes=("","_u"))

# Join with submissions using an OUTER join so that we keep the Users who do
# not have a submission
left_key = ["SourceSystemIdentifier", "LMSUserSourceSystemIdentifier"]
right_key = ["AssignmentSourceSystemIdentifier", "LMSUserSourceSystemIdentifier"]
working = working.merge(submissions, how="left", left_on=left_key, right_on=right_key, suffixes=("", "_sub"))

# How many rows _do_ have submissions?
count_submitted = working[working["SourceSystemIdentifier_sub"].notnull()].shape[0]
display(Markdown(f"Number of assignments submitted: {count_submitted}"))

# Now drop all rows that _do_ have a completed submission, which can be done easily
# by creating a new DataFrame that targets those where the submission does not
# exist or it does not have status of "Returned" or "Late".
no_submission = working[working["SourceSystemIdentifier_sub"].isnull()]
no_submission = no_submission.append(working[working["SubmissionStatus"] == "Created"])

# Might be nice to report the section name too
left_key = "LMSSectionSourceSystemIdentifier"
right_key = "SourceSystemIdentifier"
no_submission = no_submission.merge(sections, left_on=left_key, right_on=right_key, suffixes=("", "_sec"))

# Finally, narrow down to the columns of interest.
no_submission = no_submission[["Name", "Title", "Title_sec"]]
no_submission.rename(columns={"Title_sec": "Section Title", "Title": "Assignment"}, inplace=True)

display(Markdown("Missing assignment submissions:"))
display(no_submission)

Number of assignments submitted: 2

Missing assignment submissions:

,Name,Assignment,Section Title
0,Stephen Caldwell,Algebra foundations,ALG-1
1,Olivia Doris Hardy,Algebra foundations,ALG-1
2,Mary Archer,Algebra foundations,ALG-1
3,Larry Mahoney,Algebra foundations,ALG-1
